In [1]:
# /content/communes-dile-de-france-au-01-janvier.shp
# /content/communes-dile-de-france-au-01-janvier.shx

In [1]:
!pip install streamlit-folium

In [3]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install streamlit

In [3]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np

from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import os
import ast
import folium
import branca.colormap as cm

from streamlit_folium import st_folium


csv_files = ['/content/mutations_d75_train_localized.csv',
             '/content/mutations_d77_train_localized.csv',
             '/content/mutations_d78_train_localized.csv',
             '/content/mutations_d91_train_localized.csv',
             '/content/mutations_d92_train_localized.csv',
             '/content/mutations_d93_train_localized.csv',
             '/content/mutations_d94_train_localized.csv',
             '/content/mutations_d95_train_localized.csv'
             ]

def combine_clean_files(csv_files, explod = 'l_codinsee') -> pd.DataFrame:

    
    list_dfs = [pd.read_csv(f) for f in csv_files]
    
    df = pd.concat(list_dfs, ignore_index=True)
    

    l_cols = [col for col in df.columns if col.startswith('l_')]
    df[l_cols] = df[l_cols].applymap(ast.literal_eval)
    df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
    
    return df.explode(explod)


df = combine_clean_files(csv_files)



idf_reg = gpd.read_file('/content/communes-dile-de-france-au-01-janvier.shp')
idf_reg['nomcom'] = idf_reg['nomcom'].str.encode('ISO-8859-1').str.decode('utf-8')
def convert_to_point(row):
    return Point(row["longitude"], row["latitude"])

df["lat_long"] = df.apply(convert_to_point, axis=1)


df_points = gpd.GeoDataFrame(df, geometry="lat_long")
df_points.crs = {'init': 'epsg:4326'}
idf_reg = gpd.GeoDataFrame(idf_reg, geometry='geometry')
df_points = df_points.to_crs(idf_reg.crs)


result = gpd.sjoin(idf_reg, df_points, how='inner', predicate='intersects')
grouped = result.groupby('nomcom').median().reset_index()
result = pd.merge(grouped[['nomcom', 'valeurfonc']], idf_reg[['nomcom', 'geometry']], how='left', on='nomcom')
dept_geo = gpd.GeoDataFrame(idf_reg.set_index('numdep')['geometry'], geometry='geometry').dissolve(by='numdep', aggfunc='sum')

m = folium.Map(location=[48.856614, 2.3522219],
                    zoom_start = 10, tiles='cartodbpositron')

colors = ['#00ae53', '#86dc76', '#daf8aa',
            '#ffe6a4', '#ff9a61', '#ee0028']
values = np.linspace(result['valeurfonc'].min(), result['valeurfonc'].max(), num=7)
rounded_vals = np.around(values / 100_000) * 100_000

colormap_dept = cm.StepColormap(
    
    colors=colors,
    vmin=min(result['valeurfonc']),
    vmax=max(result['valeurfonc']),
    index=rounded_vals)

style_function = lambda x: {
    'fillColor': colormap_dept(x['properties']['valeurfonc']),
    'color': '',
    'weight': 0.0001,
    'fillOpacity': 0.6
}

folium.GeoJson(
    dept_geo,
    style_function = lambda x: {
        'color': 'black',
        'weight': 2.5,
        'fillOpacity': 0
    },
    name='Departement').add_to(m)



title_html = """
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 0px; top: 0px;'>
     
<div class='legend-title'>Visualization of price of mutation per communes & arrondissements </div>
<div class='legend-scale'><font size="3">Per commune / Île-de-France / between 2014 and 2021</font></div>
</div>
"""

legend_html = "<div id='maplegend' class='maplegend' style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8); border-radius:6px; padding: 10px; font-size:14px; right: 20px; top: 20px;'>"
legend_html += "<div class='legend-title'>Valeur fonciere</div>"
legend_html += "<div class='legend-scale'>"
legend_html += "<ul class='legend-labels'>"
legend_html += "<li><span style='background:{0};opacity:0.7;'></span> < {1}k € </li>".format(colors[0], int(rounded_vals[1]/1000))
for i in range(1, len(values) - 2):
    legend_html += "<li><span style='background:{0};opacity:0.7;'></span>{1}k € - {2}k €</li>".format(colors[i], int(rounded_vals[i]/1000), int(rounded_vals[i+1]/1000))
legend_html += "<li><span style='background:{0};opacity:0.7;'></span> > {1}k € </li>".format(colors[i+1], int(rounded_vals[i+1]/1000))
legend_html += """</ul>
</div>
</div>
<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>"""

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(legend_html))

folium.GeoJson(
    gpd.GeoDataFrame(result),
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['nomcom', 'valeurfonc'],
        aliases=['Commune/Arrondisement', 'Valeur fonciere'],
        localize=False
    ),
    name='Community').add_to(m)

with st.container():
    st.subheader("Hello! We are Group 5:")
    st.title("The Right Price - data commercial proposal with Eleven")
    st.write(
        " bla bla"
    )
    

with st.container():
    st.write("---")
    left_column, right_column = st.columns(2)
    with left_column:
        st.header("Try it yourself")
        st.write("##")
        st.write(
            """
            we truly believe that being the potential client a real estate developer willing to understand the market today, we can differentiate our selves focusing on allowing them to make profit so
            Focus on under/overvaluation
            Under/overvaluation is the result of 2 factors : an estimation of the price at time T, and the possibility to resell in the future (taking into account things such as RE growth rate, macro economic indicators, societal trends, financial indicators)] 
            """
        )


    with right_column:
        st.header("Map")
        st.write("these are some of the mutations")

        address = st.number_input('Insert a number')
        st.write('The current number is ', address)


        df = pd.DataFrame(
            np.random.randn(100, 2) / [40, 40] + [48.86, 2.35],
            columns=['lat', 'lon'])

        st.map(df)

    #kun's heatmap
    with st.container():
        st.header("Heatmap of value of different zones")
        st.write('The heatmap illustrates the magnitude of the value in money terms of the areas')
        map_heat = st_folium(m, width=724) 
        


    with st.container():
        st.header("Please input on the side bar the location you want to inspect")
        st.write('you will get a detailed report of the caracteristic of the mutations in the area around it')

        # default cafe de flore, saint germain de pres

        street = st.sidebar.text_input("Street", "172 Bd Saint-Germain")
        city = st.sidebar.text_input("City", "Paris")
        province = st.sidebar.text_input("Province", "Ile de france")
        country = st.sidebar.text_input("Country", "France")

        geolocator = Nominatim(user_agent="GTA Lookup")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        location = geolocator.geocode(street+", "+city+", "+province+", "+country)

        lat = location.latitude
        lon = location.longitude

        map_data = pd.DataFrame({'lat': [lat], 'lon': [lon]})

        st.map(map_data) 

Writing app.py


In [4]:
!npm install localtunnel
!pip install pyngrok

'npm' is not recognized as an internal or external command,
operable program or batch file.


  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19796 sha256=d7fa971df573cee132a26b932518a2b4ad2889ed2780494ef3905a84c230260e
  Stored in directory: c:\users\ckunt\appdata\local\pip\cache\wheels\f6\89\59\49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [5]:
from pyngrok import ngrok

In [6]:
!nohup streamlit run app.py &
url = ngrok.connect(port = '8501')
#print(url)

OSError: Background processes not supported.

In [7]:
!streamlit run app.py & npx localtunnel --port 8501

^C
